In [1]:
# http://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

In [2]:
from piper.defaults import *

piper v0.1.0: Monday, 29 March 2021 19:05:45


In [3]:
def set_categories(df):
    
    categories = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority']

    for category in categories:   
        categories = count(df, category).index.tolist()
        categories = pd.CategoricalDtype(categories)
        df[category] = df[category].astype(categories)
        memory(df)
        
    return df

In [4]:
def format_dates(df):
    
    logger.info('formatting dates...')
    
    dates = ['Order Date', 'Ship Date']
    
    for date_ in dates:
        df[date_] = pd.to_datetime(df[date_], format='%m/%d/%Y')
        
    return df

In [5]:
xl_file = 'inputs/1000000 Sales Records.csv'

In [6]:
%%piper
df <- pd.read_csv(xl_file)
>> drop(columns=['Total Revenue', 'Total Cost', 'Total Profit'])
>> distinct() # remove duplicate rows
>> format_dates()
>> set_categories() 

formatting dates...


Dataframe consumes 288.0 Mb


Dataframe consumes 227.07 Mb


Dataframe consumes 165.48 Mb


Dataframe consumes 105.88 Mb


Dataframe consumes 51.52 Mb


In [7]:
duration_limit = pd.Timedelta(50, unit='days')

In [8]:
%%piper 
df
>> clean_columns()
>> assign(lead_time=lambda x: x['ship_date'] - x['order_date'])
>> relocate('order_id', 'after', 'order_priority')
>> relocate('lead_time', 'after', 'ship_date')
>> where("lead_time >= @duration_limit", global_dict=globals())
>> head(5)

19697 rows, 12 columns


,region,country,item_type,sales_channel,order_priority,order_id,order_date,ship_date,lead_time,units_sold,unit_price,unit_cost
22,Asia,Turkmenistan,Vegetables,Online,M,116205585,2015-04-13,2015-06-02,50 days,6670,154.06,90.93
29,Europe,Estonia,Household,Offline,H,835696351,2011-09-01,2011-10-21,50 days,9976,668.27,502.54
103,Middle East and North Africa,United Arab Emirates,Baby Food,Offline,M,278474080,2017-04-25,2017-06-14,50 days,5523,255.28,159.42
128,Asia,Mongolia,Meat,Offline,H,177994239,2014-02-07,2014-03-29,50 days,1306,421.89,364.69
218,Asia,Kazakhstan,Personal Care,Offline,L,210387599,2013-01-25,2013-03-16,50 days,7221,81.73,56.67


In [9]:
%piper df >> info()

Dataframe consumes 51.52 Mb


,columns,type,n,isna,isnull,unique
0,region,category,1000000,0,0,7
1,country,category,1000000,0,0,185
2,item_type,category,1000000,0,0,12
3,sales_channel,category,1000000,0,0,2
4,order_priority,category,1000000,0,0,4
5,order_date,datetime64[ns],1000000,0,0,2767
6,order_id,int64,1000000,0,0,900000
7,ship_date,datetime64[ns],1000000,0,0,2817
8,units_sold,int64,1000000,0,0,10000
9,unit_price,float64,1000000,0,0,12


In [10]:
%%piper 
count(df, 'Region', percent=True, reset_index=True)

Column Region not found!


In [11]:
%%piper 
df 
>> count('sales_channel', reset_index=True)

,sales_channel,n,%,cum %
0,Offline,500249,50.02,50.02
1,Online,499751,49.98,100.00


In [12]:
%%piper
df 
>> group_by(['region', 'order_priority'])
>> summarise(totalunitssold=('units_sold', 'sum'))
>> unstack()
>> flatten_cols(remove_prefix='totalunitssold')
>> adorn(axis='column')
>> reset_index()
>> order_by('-All')
>> adorn(ignore_index=True)
>> set_columns(columns=['Region', 'C', 'H', 'M', 'L', 'All'])

,Region,C,H,M,L,All
0,Sub-Saharan Africa,325811341,323894262,323777790,326324159,1299807552
1,Europe,324250451,323206262,324086079,324340380,1295883172
2,Asia,182384174,182027601,182127329,181967072,728506176
3,Middle East and North Africa,154929315,154570759,155759219,156089279,621348572
4,Central America and the Caribbean,134136229,134876690,136222257,134614184,539849360
5,Australia and Oceania,99490384,102129538,101751778,101670417,405042117
6,North America,26874028,27300491,27232388,27023446,108430353
7,All,1247875922,1248005603,1250956840,1252028937,4998867302
